# 第06章：Document Processing（文档处理）

## 学习目标

本章将学习：
1. Document对象的结构和作用
2. 使用Document Loaders加载各种格式的文档
3. 使用Text Splitters将文档分割成小块
4. 理解chunk size和chunk overlap的作用
5. 掌握RecursiveCharacterTextSplitter的使用

## 为什么需要Document Processing？

在构建RAG系统时，我们需要：
- 从各种来源加载文档（PDF、网页、数据库等）
- 将大文档分割成小块，以便：
  - 适应模型的上下文窗口限制
  - 提高检索的精确度
  - 避免无关信息干扰

Document Processing是RAG流程的第一步，也是基础。

In [ ]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.messages import HumanMessage, AIMessage, ToolMessage

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,  # 工具调用建议使用低温度
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成")
print(f"模型: {model.model_name}")

环境配置完成
模型: gpt-4.1-mini


## 2. Document对象

Document是LangChain中表示文档的核心抽象。

### Document的结构

一个Document对象包含三个主要属性：
- `page_content`：文档的文本内容（字符串）
- `metadata`：文档的元数据（字典），可以包含来源、页码、作者等信息
- `id`：可选的文档标识符（字符串）

Document对象通常代表一个大文档的一个片段（chunk）。


In [2]:
from langchain_core.documents import Document

print("【创建Document对象】")
print()

# 创建示例文档
documents = [
    Document(
        page_content="狗是很好的伙伴，以忠诚和友好而闻名。",
        metadata={"source": "宠物文档", "category": "哺乳动物"}
    ),
    Document(
        page_content="猫是独立的宠物，经常喜欢自己的空间。",
        metadata={"source": "宠物文档", "category": "哺乳动物"}
    ),
    Document(
        page_content="Python是一种高级编程语言，以其简洁的语法而闻名。",
        metadata={"source": "编程文档", "category": "技术", "difficulty": "beginner"}
    ),
]

# 查看Document对象
for i, doc in enumerate(documents, 1):
    print(f"文档 {i}:")
    print(f"  内容: {doc.page_content}")
    print(f"  元数据: {doc.metadata}")
    print(f"  ID: {doc.id}")
    print()

【创建Document对象】

文档 1:
  内容: 狗是很好的伙伴，以忠诚和友好而闻名。
  元数据: {'source': '宠物文档', 'category': '哺乳动物'}
  ID: None

文档 2:
  内容: 猫是独立的宠物，经常喜欢自己的空间。
  元数据: {'source': '宠物文档', 'category': '哺乳动物'}
  ID: None

文档 3:
  内容: Python是一种高级编程语言，以其简洁的语法而闻名。
  元数据: {'source': '编程文档', 'category': '技术', 'difficulty': 'beginner'}
  ID: None



## 3. Document Loaders（文档加载器）

Document Loaders用于从各种数据源加载文档，并将其转换为统一的Document对象。

### 常见的Document Loaders

LangChain提供了数百种Document Loaders，支持：
- 文件格式：PDF、CSV、JSON、TXT、DOCX、EPUB、PPTX等
- 数据源：Google Drive、Slack、Notion、数据库等
- 网络内容：网页、API等

### Document Loader的标准接口

所有Document Loaders都实现统一的接口：
- `load()`：一次性加载所有文档
- `load_and_split()`：加载并自动分割文档


In [3]:
### 3.1 加载文本文件

print("【示例1：加载文本文件】")
print()

# 创建一个示例文本文件
with open("../data/sample.txt", "w", encoding="utf-8") as f:
    f.write("LangChain是一个用于构建基于语言模型应用的框架。\n")
    f.write("它提供了强大的工具来处理文档、构建RAG系统和创建Agent。\n")
    f.write("LangChain支持多种语言模型提供商，包括OpenAI、Anthropic等。\n")

from langchain_community.document_loaders import TextLoader

# 加载文本文件
loader = TextLoader("../data/sample.txt", encoding="utf-8")
docs = loader.load()

print(f"加载了 {len(docs)} 个文档")
print()
print("文档内容:")
print(f"  page_content: {docs[0].page_content}")
print(f"  metadata: {docs[0].metadata}")
print()

【示例1：加载文本文件】

加载了 1 个文档

文档内容:
  page_content: LangChain是一个用于构建基于语言模型应用的框架。
它提供了强大的工具来处理文档、构建RAG系统和创建Agent。
LangChain支持多种语言模型提供商，包括OpenAI、Anthropic等。

  metadata: {'source': '../data/sample.txt'}



In [4]:
### 3.2 加载CSV文件

print("【示例2：加载CSV文件】")
print()

# 创建示例CSV文件
import csv

with open("../data/products.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["产品名称", "价格", "描述"])
    writer.writerow(["笔记本电脑", "5999", "高性能办公笔记本"])
    writer.writerow(["鼠标", "99", "无线蓝牙鼠标"])
    writer.writerow(["键盘", "299", "机械键盘"])

from langchain_community.document_loaders import CSVLoader

# 加载CSV文件
loader = CSVLoader("../data/products.csv")
docs = loader.load()

print(f"加载了 {len(docs)} 个文档")
print()
for i, doc in enumerate(docs[:2], 1):
    print(f"文档 {i}:")
    print(f"  内容: {doc.page_content[:100]}...")
    print(f"  元数据: {doc.metadata}")
    print()

【示例2：加载CSV文件】

加载了 3 个文档

文档 1:
  内容: 产品名称: 笔记本电脑
价格: 5999
描述: 高性能办公笔记本...
  元数据: {'source': '../data/products.csv', 'row': 0}

文档 2:
  内容: 产品名称: 鼠标
价格: 99
描述: 无线蓝牙鼠标...
  元数据: {'source': '../data/products.csv', 'row': 1}



In [5]:
### 3.3 加载JSON文件

print("【示例3：加载JSON文件】")
print()

# 创建示例JSON文件
import json

data = {
    "users": [
        {"name": "张三", "age": 25, "city": "北京"},
        {"name": "李四", "age": 30, "city": "上海"},
        {"name": "王五", "age": 28, "city": "深圳"}
    ]
}

with open("../data/users.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

from langchain_community.document_loaders import JSONLoader

# 加载JSON文件（提取users数组中的每个对象）
loader = JSONLoader(
    file_path="../data/users.json",
    jq_schema=".users[]",
    text_content=False
)
docs = loader.load()

print(f"加载了 {len(docs)} 个文档")
print()
for i, doc in enumerate(docs, 1):
    print(f"文档 {i}:")
    # 将转义的字符串重新解码
    content_obj = json.loads(doc.page_content)
    print(f"  内容: {json.dumps(content_obj, ensure_ascii=False, indent=2)}")
    print(f"  元数据: {doc.metadata}")
    print()


【示例3：加载JSON文件】

加载了 3 个文档

文档 1:
  内容: {
  "name": "张三",
  "age": 25,
  "city": "北京"
}
  元数据: {'source': '/home/iip/tp/038-project/rag/data/users.json', 'seq_num': 1}

文档 2:
  内容: {
  "name": "李四",
  "age": 30,
  "city": "上海"
}
  元数据: {'source': '/home/iip/tp/038-project/rag/data/users.json', 'seq_num': 2}

文档 3:
  内容: {
  "name": "王五",
  "age": 28,
  "city": "深圳"
}
  元数据: {'source': '/home/iip/tp/038-project/rag/data/users.json', 'seq_num': 3}



## 4. Text Splitters（文本分割器）

### 为什么需要Text Splitters？

当我们加载文档后，通常需要将其分割成更小的块，原因包括：

1. **上下文窗口限制**：大多数模型都有上下文窗口限制（如4K、8K tokens）
2. **检索精确度**：小块更容易匹配具体的查询
3. **成本优化**：只传递相关部分给模型，减少token消耗
4. **语义完整性**：避免无关信息干扰模型理解

### 分割策略

LangChain提供多种分割策略：

1. **基于长度的分割**：
   - CharacterTextSplitter：按字符数分割
   - TokenTextSplitter：按token数分割

2. **基于文本结构的分割**（推荐）：
   - RecursiveCharacterTextSplitter：递归使用多种分隔符
   - 保持段落、句子的完整性

3. **基于文档结构的分割**：
   - MarkdownTextSplitter：按Markdown标题分割
   - HTMLTextSplitter：按HTML标签分割
   - CodeTextSplitter：按代码结构分割


In [6]:
### 4.1 CharacterTextSplitter（基础分割器）

print("【示例1：CharacterTextSplitter】")
print()

from langchain_text_splitters import CharacterTextSplitter

# 创建一个较长的文本
text = """
LangChain 核心功能详解

LangChain 是一个用于构建大语言模型（LLM）应用的开源框架，它提供了丰富的工具和组件。

复我也是真的非常的醉了，可惜了，真的很不错。

杂非常复杂真是复杂极了非常开心

的

AI

应

用

。

核

心

模

块

包

括

：

这

些

1. 模型集成（Model I/O）：支持对接OpenAI、Anthropic、百度文心等主流LLM，提供统一的调用接口，简化模型切换和参数管理。

2. 数据连接（Data Connection）：提供文档加载、文本分割、向量存储等功能，支持RAG（检索增强生成）场景，让LLM能结合外部数据回答问题。
3. 链（Chains）：将多个组件组合成流水线，比如“加载文档→分割文本→向量检索→生成回答”，实现端到端的AI应用流程。
4. 智能代理（Agents）：让LLM能自主决策调用哪些工具（如计算器、搜索引擎），完成复杂任务，比如“分析财报并生成可视化图表”。

LangChain的优势在于灵活性和扩展性：开发者可以按需组合组件，也可以自定义组件适配特定场景；同时它支持多种部署方式，包括本地部署、云部署、边缘部署等，满足不同的生产环境需求。
在实际应用中，LangChain常被用于构建智能客服、文档问答系统、代码生成工具、数据分析助手等，是当前LLM应用开发的主流框架之一。
"""

# 创建分割器
splitter = CharacterTextSplitter(
    separator="\n\n",  # 使用双换行符作为分隔符
    chunk_size=100,    # 每块最大100字符
    chunk_overlap=20,  # 块之间重叠20字符
)

# 分割文本
chunks = splitter.split_text(text)

print(f"文本被分割成 {len(chunks)} 个块")
print()
for i, chunk in enumerate(chunks, 1):
    print(f"块 {i} ({len(chunk)} 字符):")
    print(f"{chunk.strip()}")
    print("-" * 60)
    print()

Created a chunk of size 208, which is longer than the specified 100


【示例1：CharacterTextSplitter】

文本被分割成 5 个块

块 1 (90 字符):
LangChain 核心功能详解

LangChain 是一个用于构建大语言模型（LLM）应用的开源框架，它提供了丰富的工具和组件。

复我也是真的非常的醉了，可惜了，真的很不错。
------------------------------------------------------------

块 2 (58 字符):
杂非常复杂真是复杂极了非常开心

的

AI

应

用

。

核

心

模

块

包

括

：

这

些
------------------------------------------------------------

块 3 (94 字符):
模

块

包

括

：

这

些

1. 模型集成（Model I/O）：支持对接OpenAI、Anthropic、百度文心等主流LLM，提供统一的调用接口，简化模型切换和参数管理。
------------------------------------------------------------

块 4 (208 字符):
2. 数据连接（Data Connection）：提供文档加载、文本分割、向量存储等功能，支持RAG（检索增强生成）场景，让LLM能结合外部数据回答问题。
3. 链（Chains）：将多个组件组合成流水线，比如“加载文档→分割文本→向量检索→生成回答”，实现端到端的AI应用流程。
4. 智能代理（Agents）：让LLM能自主决策调用哪些工具（如计算器、搜索引擎），完成复杂任务，比如“分析财报并生成可视化图表”。
------------------------------------------------------------

块 5 (158 字符):
LangChain的优势在于灵活性和扩展性：开发者可以按需组合组件，也可以自定义组件适配特定场景；同时它支持多种部署方式，包括本地部署、云部署、边缘部署等，满足不同的生产环境需求。
在实际应用中，LangChain常被用于构建智能客服、文档问答系统、代码生成工具、数据分析助手等，是当前LLM应用开发的主流框架之一。
------------------------------

### 4.2 RecursiveCharacterTextSplitter（推荐）

RecursiveCharacterTextSplitter是官方推荐的通用文本分割器。

#### 工作原理

它会递归地尝试使用以下分隔符（按顺序）：
1. `\n\n`（双换行，段落）
2. `\n`（单换行，行）
3. ` `（空格，单词）
4. `""`（字符）

这样可以尽可能保持文本结构的完整性。


In [7]:
print("【示例2：RecursiveCharacterTextSplitter】")
print()

from langchain_text_splitters import RecursiveCharacterTextSplitter

# 创建一个包含多种结构的文本
text = """# LangChain学习指南

## 什么是LangChain？

LangChain是一个用于构建基于语言模型应用的框架。它提供了一系列工具和抽象，使开发者能够轻松地：
- 连接多个组件
- 管理提示词
- 调用语言模型
- 处理输出

## 核心概念

### Models
Models是LangChain的基础。LangChain支持各种语言模型提供商。

### Prompts
Prompts用于指导模型的行为。好的prompt可以显著提高模型的性能。

### Chains
Chains允许你将多个组件组合在一起，创建复杂的应用。
"""

# 创建递归分割器
splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,     # 每块最大150字符
    chunk_overlap=30,   # 块之间重叠30字符
    length_function=len,
)

# 分割文本
chunks = splitter.split_text(text)

print(f"文本被分割成 {len(chunks)} 个块")
print()
for i, chunk in enumerate(chunks, 1):
    print(f"块 {i} ({len(chunk)} 字符):")
    print(chunk.strip())
    print("=" * 60)
    print()

【示例2：RecursiveCharacterTextSplitter】

文本被分割成 3 个块

块 1 (128 字符):
# LangChain学习指南

## 什么是LangChain？

LangChain是一个用于构建基于语言模型应用的框架。它提供了一系列工具和抽象，使开发者能够轻松地：
- 连接多个组件
- 管理提示词
- 调用语言模型
- 处理输出

## 核心概念

块 2 (112 字符):
## 核心概念

### Models
Models是LangChain的基础。LangChain支持各种语言模型提供商。

### Prompts
Prompts用于指导模型的行为。好的prompt可以显著提高模型的性能。

块 3 (39 字符):
### Chains
Chains允许你将多个组件组合在一起，创建复杂的应用。



In [8]:
### 4.3 分割Document对象

print("【示例3：分割Document对象】")
print()

# 创建一个长文档
long_doc = Document(
    page_content="""
LangChain是一个强大的框架，用于构建基于语言模型的应用。它提供了丰富的工具和抽象层。

RAG（检索增强生成）是LangChain的核心应用场景之一。通过RAG，我们可以让模型访问外部知识库。

Agent是LangChain的另一个重要特性。Agent能够自主决定使用哪些工具来完成任务。

LangGraph则是用于构建复杂多智能体系统的工具，它基于状态机的概念。
    """,
    metadata={"source": "LangChain教程", "chapter": 1}
)

# 使用split_documents方法
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    add_start_index=True,  # 记录每个块在原文中的起始位置
)

# 分割文档
split_docs = text_splitter.split_documents([long_doc])

print(f"原始文档被分割成 {len(split_docs)} 个子文档")
print()
for i, doc in enumerate(split_docs, 1):
    print(f"子文档 {i}:")
    print(f"  内容: {doc.page_content.strip()}")
    print(f"  元数据: {doc.metadata}")
    print()

【示例3：分割Document对象】

原始文档被分割成 3 个子文档

子文档 1:
  内容: LangChain是一个强大的框架，用于构建基于语言模型的应用。它提供了丰富的工具和抽象层。
  元数据: {'source': 'LangChain教程', 'chapter': 1, 'start_index': 1}

子文档 2:
  内容: RAG（检索增强生成）是LangChain的核心应用场景之一。通过RAG，我们可以让模型访问外部知识库。
  元数据: {'source': 'LangChain教程', 'chapter': 1, 'start_index': 49}

子文档 3:
  内容: Agent是LangChain的另一个重要特性。Agent能够自主决定使用哪些工具来完成任务。

LangGraph则是用于构建复杂多智能体系统的工具，它基于状态机的概念。
  元数据: {'source': 'LangChain教程', 'chapter': 1, 'start_index': 103}



## 5. 关键参数详解

### 5.1 chunk_size（块大小）

chunk_size决定每个文本块的最大长度。

#### 如何选择chunk_size？

| chunk_size | 适用场景 | 优点 | 缺点 |
|-----------|---------|------|------|
| 小（100-300） | 精确匹配、问答 | 检索精确 | 可能丢失上下文 |
| 中（500-1000） | 通用RAG（推荐） | 平衡性好 | - |
| 大（1500-2000） | 需要完整上下文 | 上下文完整 | 检索不够精确 |

#### 影响因素

1. **模型上下文窗口**：不能超过模型限制
2. **内容类型**：代码、文章、对话等需要不同的大小
3. **检索需求**：需要精确匹配还是完整上下文


### 5.2 chunk_overlap（块重叠）

chunk_overlap决定相邻文本块之间的重叠字符数。

#### 为什么需要重叠？

重叠可以解决以下问题：
1. **避免语句被截断**：重要信息可能横跨两个块的边界
2. **保持上下文连贯性**：让每个块都有前后文
3. **提高检索召回率**：同一信息可能出现在多个块中

#### 如何选择chunk_overlap？

一般建议：
- chunk_overlap = chunk_size * 10%-20%
- 例如：chunk_size=1000，则chunk_overlap=100-200

过大的重叠会导致：
- 存储空间浪费
- 检索时重复内容过多

过小的重叠会导致：
- 重要信息被截断
- 上下文不完整


In [9]:
print("【对比：有无重叠的区别】")
print()

text = "人工智能正在改变世界。机器学习是人工智能的核心技术。深度学习推动了AI的快速发展。"

# 无重叠
splitter_no_overlap = RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=0,
)
chunks_no_overlap = splitter_no_overlap.split_text(text)

print("无重叠:")
for i, chunk in enumerate(chunks_no_overlap, 1):
    print(f"  块{i}: {chunk}")
print()

# 有重叠
splitter_with_overlap = RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=10,
)
chunks_with_overlap = splitter_with_overlap.split_text(text)

print("有重叠（overlap=10）:")
for i, chunk in enumerate(chunks_with_overlap, 1):
    print(f"  块{i}: {chunk}")
print()

print("注意：")
print("- 无重叠：每个块之间完全独立，可能导致信息被截断")
print("- 有重叠：块之间有共同部分，保证了上下文的连续性")


【对比：有无重叠的区别】

无重叠:
  块1: 人工智能正在改变世界。机器学习是人工智能的核心技术。深度学习
  块2: 推动了AI的快速发展。

有重叠（overlap=10）:
  块1: 人工智能正在改变世界。机器学习是人工智能的核心技术。深度学习
  块2: 的核心技术。深度学习推动了AI的快速发展。

注意：
- 无重叠：每个块之间完全独立，可能导致信息被截断
- 有重叠：块之间有共同部分，保证了上下文的连续性


## 6. 完整流程示例

现在让我们将Document Loader和Text Splitter结合起来，完成一个完整的文档处理流程。


In [ ]:
print("【完整的文档处理流程】")
print()

# 步骤1：创建一个较长的文档
with open("../data/langchain_intro.txt", "w", encoding="utf-8") as f:
    f.write("""LangChain简介

LangChain是一个强大的框架，专门用于构建基于大型语言模型（LLM）的应用程序。它提供了一套完整的工具链，帮助开发者快速构建智能应用。

核心特性

1. 模型集成
LangChain支持多种语言模型提供商，包括OpenAI、Anthropic、Google等。开发者可以轻松切换不同的模型。

2. 提示词管理
LangChain提供了强大的提示词模板系统，支持动态变量、条件逻辑和少样本学习。

3. 链式调用
通过LCEL（LangChain Expression Language），可以将多个组件链接在一起，构建复杂的处理流程。

4. RAG支持
LangChain内置了完整的RAG工具链，包括文档加载器、文本分割器、向量数据库集成等。

5. Agent框架
LangChain提供了Agent框架，使模型能够自主决策和使用工具。

应用场景

- 问答系统
- 文档分析
- 代码生成
- 对话机器人
- 数据提取
- 智能搜索

总结

LangChain是构建LLM应用的最佳选择，它大大降低了开发难度，提高了开发效率。
""")

# 步骤2：加载文档
print("步骤1：加载文档")
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/langchain_intro.txt", encoding="utf-8")
docs = loader.load()

print(f"  加载了 {len(docs)} 个文档")
print(f"  文档长度: {len(docs[0].page_content)} 字符")
print()

# 步骤3：分割文档
print("步骤2：分割文档")
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 每块200字符
    chunk_overlap=50,    # 重叠50字符
    add_start_index=True # 记录起始位置
)

split_docs = text_splitter.split_documents(docs)

print(f"  分割成 {len(split_docs)} 个块")
print()

# 步骤4：查看分割结果
print("步骤3：查看前3个块")
print()
for i, doc in enumerate(split_docs[:3], 1):
    print(f"块 {i}:")
    print(f"  长度: {len(doc.page_content)} 字符")
    print(f"  起始位置: {doc.metadata.get('start_index', 'N/A')}")
    print(f"  内容: {doc.page_content[:100]}...")
    print()

print("步骤4：验证重叠")
print()
# 查看第1块和第2块的重叠部分
if len(split_docs) >= 2:
    chunk1 = split_docs[0].page_content
    chunk2 = split_docs[1].page_content
    
    # 找出重叠部分（简化版）
    overlap_found = False
    for i in range(len(chunk1)):
        if chunk2.startswith(chunk1[i:]):
            overlap = chunk1[i:]
            print(f"块1和块2的重叠部分（{len(overlap)}字符）:")
            print(f"  '{overlap}'")
            overlap_found = True
            break
    
    if not overlap_found:
        print("  未发现明显重叠（可能因为分割点在特殊位置）")

print()
print("处理完成！这些分割后的文档现在可以用于：")
print("- 生成向量嵌入（Embeddings）")
print("- 存储到向量数据库")
print("- 用于RAG检索")

【完整的文档处理流程】

步骤1：加载文档
  加载了 1 个文档
  文档长度: 488 字符

步骤2：分割文档
  分割成 3 个块

步骤3：查看前3个块

块 1:
  长度: 162 字符
  起始位置: 0
  内容: LangChain简介

LangChain是一个强大的框架，专门用于构建基于大型语言模型（LLM）的应用程序。它提供了一套完整的工具链，帮助开发者快速构建智能应用。

核心特性

1. 模型集成
L...

块 2:
  长度: 177 字符
  起始位置: 164
  内容: 2. 提示词管理
LangChain提供了强大的提示词模板系统，支持动态变量、条件逻辑和少样本学习。

3. 链式调用
通过LCEL（LangChain Expression Language），可以...

块 3:
  长度: 144 字符
  起始位置: 343
  内容: 5. Agent框架
LangChain提供了Agent框架，使模型能够自主决策和使用工具。

应用场景

- 问答系统
- 文档分析
- 代码生成
- 对话机器人
- 数据提取
- 智能搜索

总结...

步骤4：验证重叠

  未发现明显重叠（可能因为分割点在特殊位置）

处理完成！这些分割后的文档现在可以用于：
- 生成向量嵌入（Embeddings）
- 存储到向量数据库
- 用于RAG检索


## 7. 实战项目：智能文档处理系统

构建一个完整的文档处理系统，支持：
1. 多种格式的文档加载
2. 智能分割策略
3. 元数据管理
4. 统计分析


In [11]:
print("【智能文档处理系统】")
print()

from typing import List, Dict
from dataclasses import dataclass

@dataclass
class ProcessingStats:
    """处理统计信息"""
    total_docs: int
    total_chunks: int
    avg_chunk_size: float
    min_chunk_size: int
    max_chunk_size: int
    file_types: Dict[str, int]

class DocumentProcessor:
    """智能文档处理器"""
    
    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        """
        初始化文档处理器
        
        Args:
            chunk_size: 块大小
            chunk_overlap: 重叠大小
        """
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            add_start_index=True
        )
        self.documents = []
        self.chunks = []
    
    def load_text(self, file_path: str) -> List[Document]:
        """加载文本文件"""
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        # 添加文件类型到元数据
        for doc in docs:
            doc.metadata["file_type"] = "text"
        return docs
    
    def load_csv(self, file_path: str) -> List[Document]:
        """加载CSV文件"""
        loader = CSVLoader(file_path)
        docs = loader.load()
        for doc in docs:
            doc.metadata["file_type"] = "csv"
        return docs
    
    def load_json(self, file_path: str, jq_schema: str = ".") -> List[Document]:
        """加载JSON文件"""
        loader = JSONLoader(
            file_path=file_path,
            jq_schema=jq_schema,
            text_content=False
        )
        docs = loader.load()
        for doc in docs:
            doc.metadata["file_type"] = "json"
        return docs
    
    def add_documents(self, docs: List[Document]):
        """添加文档到处理器"""
        self.documents.extend(docs)
    
    def process(self):
        """处理所有文档：分割成块"""
        self.chunks = self.text_splitter.split_documents(self.documents)
        return self.chunks
    
    def get_stats(self) -> ProcessingStats:
        """获取处理统计信息"""
        if not self.chunks:
            return None
        
        chunk_sizes = [len(chunk.page_content) for chunk in self.chunks]
        
        # 统计文件类型
        file_types = {}
        for doc in self.documents:
            file_type = doc.metadata.get("file_type", "unknown")
            file_types[file_type] = file_types.get(file_type, 0) + 1
        
        return ProcessingStats(
            total_docs=len(self.documents),
            total_chunks=len(self.chunks),
            avg_chunk_size=sum(chunk_sizes) / len(chunk_sizes),
            min_chunk_size=min(chunk_sizes),
            max_chunk_size=max(chunk_sizes),
            file_types=file_types
        )
    
    def search_chunks(self, keyword: str) -> List[Document]:
        """搜索包含关键词的块"""
        return [
            chunk for chunk in self.chunks 
            if keyword.lower() in chunk.page_content.lower()
        ]
    
    def filter_by_metadata(self, key: str, value: str) -> List[Document]:
        """根据元数据筛选块"""
        return [
            chunk for chunk in self.chunks
            if chunk.metadata.get(key) == value
        ]

print("文档处理器类定义完成！")

【智能文档处理系统】

文档处理器类定义完成！


In [12]:
print("【使用文档处理系统】")
print()

# 1. 创建处理器
processor = DocumentProcessor(chunk_size=300, chunk_overlap=50)

print("步骤1：加载多种格式的文档")
# 加载文本文件
text_docs = processor.load_text("../data/langchain_intro.txt")
processor.add_documents(text_docs)
print(f"  加载文本文件: {len(text_docs)} 个文档")

# 加载CSV文件
csv_docs = processor.load_csv("../data/products.csv")
processor.add_documents(csv_docs)
print(f"  加载CSV文件: {len(csv_docs)} 个文档")

# 加载JSON文件
json_docs = processor.load_json("../data/users.json", jq_schema=".users[]")
processor.add_documents(json_docs)
print(f"  加载JSON文件: {len(json_docs)} 个文档")

print(f"  总计: {len(processor.documents)} 个文档")
print()

# 2. 处理文档
print("步骤2：分割文档")
chunks = processor.process()
print(f"  生成了 {len(chunks)} 个文本块")
print()

# 3. 获取统计信息
print("步骤3：统计信息")
stats = processor.get_stats()
print(f"  原始文档数: {stats.total_docs}")
print(f"  文本块数: {stats.total_chunks}")
print(f"  平均块大小: {stats.avg_chunk_size:.1f} 字符")
print(f"  最小块大小: {stats.min_chunk_size} 字符")
print(f"  最大块大小: {stats.max_chunk_size} 字符")
print(f"  文件类型分布: {stats.file_types}")
print()

# 4. 搜索功能
print("步骤4：搜索功能")
search_results = processor.search_chunks("LangChain")
print(f"  搜索'LangChain'找到 {len(search_results)} 个块")
if search_results:
    print(f"  第一个结果: {search_results[0].page_content[:100]}...")
print()

# 5. 元数据筛选
print("步骤5：元数据筛选")
text_chunks = processor.filter_by_metadata("file_type", "text")
print(f"  文本文件的块数: {len(text_chunks)}")

csv_chunks = processor.filter_by_metadata("file_type", "csv")
print(f"  CSV文件的块数: {len(csv_chunks)}")

json_chunks = processor.filter_by_metadata("file_type", "json")
print(f"  JSON文件的块数: {len(json_chunks)}")
print()

# 6. 查看示例块
print("步骤6：查看示例块（来自不同文件类型）")
print()

for file_type in ["text", "csv", "json"]:
    chunks_of_type = processor.filter_by_metadata("file_type", file_type)
    if chunks_of_type:
        chunk = chunks_of_type[0]
        print(f"{file_type.upper()} 文件示例:")
        print(f"  内容: {chunk.page_content[:150]}...")
        print(f"  元数据: {chunk.metadata}")
        print()

【使用文档处理系统】

步骤1：加载多种格式的文档
  加载文本文件: 1 个文档
  加载CSV文件: 3 个文档
  加载JSON文件: 3 个文档
  总计: 7 个文档

步骤2：分割文档
  生成了 8 个文本块

步骤3：统计信息
  原始文档数: 7
  文本块数: 8
  平均块大小: 93.2 字符
  最小块大小: 25 字符
  最大块大小: 285 字符
  文件类型分布: {'text': 1, 'csv': 3, 'json': 3}

步骤4：搜索功能
  搜索'LangChain'找到 2 个块
  第一个结果: LangChain简介

LangChain是一个强大的框架，专门用于构建基于大型语言模型（LLM）的应用程序。它提供了一套完整的工具链，帮助开发者快速构建智能应用。

核心特性

1. 模型集成
L...

步骤5：元数据筛选
  文本文件的块数: 2
  CSV文件的块数: 3
  JSON文件的块数: 3

步骤6：查看示例块（来自不同文件类型）

TEXT 文件示例:
  内容: LangChain简介

LangChain是一个强大的框架，专门用于构建基于大型语言模型（LLM）的应用程序。它提供了一套完整的工具链，帮助开发者快速构建智能应用。

核心特性

1. 模型集成
LangChain支持多种语言模型提供商，包括OpenAI、Anthropic、Google等。开发者...
  元数据: {'source': '../data/langchain_intro.txt', 'file_type': 'text', 'start_index': 0}

CSV 文件示例:
  内容: 产品名称: 笔记本电脑
价格: 5999
描述: 高性能办公笔记本...
  元数据: {'source': '../data/products.csv', 'row': 0, 'file_type': 'csv', 'start_index': 0}

JSON 文件示例:
  内容: {"name": "\u5f20\u4e09", "age": 25, "city": "\u5317\u4eac"}...
  元数据: {'source': '/home/iip/tp/038-project/rag/data/users.js

## 8. 总结与最佳实践

### 核心概念回顾

1. **Document对象**
   - `page_content`：文本内容
   - `metadata`：元数据
   - `id`：可选标识符

2. **Document Loaders**
   - 统一接口：`load()` 和 `load_and_split()`
   - 支持多种格式：TXT、CSV、JSON、PDF、DOCX等
   - 返回标准的Document对象

3. **Text Splitters**
   - RecursiveCharacterTextSplitter（推荐）
   - 关键参数：chunk_size、chunk_overlap
   - 保持文本结构的完整性

### 最佳实践

#### 1. 选择合适的chunk_size

| 应用场景 | 推荐大小 | 说明 |
|---------|---------|------|
| 问答系统 | 500-1000 | 平衡精确度和上下文 |
| 语义搜索 | 200-500 | 更精确的匹配 |
| 文档摘要 | 1000-2000 | 需要更多上下文 |
| 代码分析 | 按函数/类分割 | 使用结构化分割器 |

#### 2. 设置合理的chunk_overlap

- 一般为chunk_size的10%-20%
- 避免重要信息被截断
- 不要过大，会浪费存储空间

#### 3. 利用元数据

- 记录文档来源
- 添加自定义标签
- 使用`add_start_index=True`跟踪位置
- 便于后续筛选和溯源

#### 4. 选择合适的分割器

- **通用文本**：RecursiveCharacterTextSplitter
- **Markdown**：MarkdownTextSplitter
- **代码**：Language-specific code splitters
- **HTML**：HTMLTextSplitter

### 常见陷阱

1. **chunk_size过大**：检索不精确，超出模型上下文窗口
2. **chunk_size过小**：丢失上下文，理解困难
3. **没有重叠**：重要信息可能被截断
4. **忽略元数据**：难以追踪来源和管理